<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250730/%E3%82%B0%E3%83%A9%E3%83%95%E5%88%86%E5%89%B2%E5%95%8F%E9%A1%8C%E3%81%AE%E6%BC%94%E7%BF%92%E5%95%8F%E9%A1%8C%E3%81%AE%E8%A7%A3%E8%AA%AC%E3%81%A8%E8%A7%A3%E7%AD%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#準備


In [ ]:
%%capture
!pip install mip
!pip install networkx
!pip install japanize-matplotlib

-----

## 演習問題 1

以下のグラフを、各区画の頂点数が2または3となるように2分割し、カットサイズを最小化せよ。辺の重みは全て1とする。

  * **頂点**: $V = {0, 1, 2, 3, 4, 5}$
  * **辺**: $E = {(0,1), (0,2), (1,2), (1,3), (2,3), (2,4), (3,4), (2,5), (3,5), (4,5)}$

###数理モデルの定式化

この問題を整数計画問題として定式化する。基本モデルは講義で解説した2分割問題の定式化を用いる。

  * **決定変数**

      * $x\_i$: 頂点 $i \\in V$ が区画1に属するならば1、区画0に属するならば0をとるバイナリ変数。
      * $z\_{uv}$: 辺 $(u,v) \\in E$ がカットされる（頂点 $u$ と $v$ が異なる区画に属する）ならば1、そうでなければ0をとるバイナリ変数。

  * **パラメータ**

      * $V$: 頂点の集合 ${0, 1, ..., 5}$。総頂点数 $N=6$。
      * $E$: 辺の集合。

  * **目的関数**
    カットされた辺の総数を最小化する。これは、カットされた辺に対応する変数 $z\_{uv}$ の合計を最小化することに等しい。

$$
\min \sum_{(u,v) \in E} z_{uv}
$$

  * **制約条件**
   1.  **カット辺の定義**: 頂点 $u$ と $v$ が異なる区画にある場合（つまり $x_u \neq x_v$）にのみ、$z_{uv}=1$ となるようにする。
$$
\begin{align}
        z_{uv} &\ge x_u - x_v \quad &\forall (u,v) \in E \\
        z_{uv} &\ge x_v - x_u \quad &\forall (u,v) \in E
\end{align}
$$
  2.  **区画サイズの制約**: 各区画の頂点数が2または3でなければならない。区画1のサイズを $\sum_{i \in V} x_i$ で表し、これが2または3になるように制約する。区画1のサイズが決まれば、区画0のサイズ ($N - \sum x_i$) も自動的に決まるため、片方の区画について制約を課せば十分である。
$$
  2 \le \sum_{i \in V} x_i \le 3
$$
    3.  **変数型制約**: 各変数がバイナリ変数であることを定義する。
$$
\begin{align}
    x_i    & \in \{0, 1\} \quad & \forall i \in V \\
    z_{uv} & \in \{0, 1\} \quad & \forall (u,v) \in E
\end{align}
$$

###Python (MIP) による実装

In [ ]:
# 必要なライブラリをインポート
import mip
import networkx as nx
import matplotlib.pyplot as plt
import japanize_matplotlib

# グラフの定義
nodes = [0, 1, 2, 3, 4, 5]
edges = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (2, 4), (3, 4), (2, 5), (3, 5), (4, 5)]
edge_weights = {edge: 1 for edge in edges} # 全ての辺の重みを1に設定

# 1. モデルの作成
model = mip.Model(name="graph_partition_1", sense=mip.MINIMIZE)

# 2. 変数の定義
# x_i: 頂点iが区画1に属するなら1
x = {i: model.add_var(var_type=mip.BINARY, name=f"x_{i}") for i in nodes}
# z_uv: 辺(u,v)がカットされるなら1
z = {(u, v): model.add_var(var_type=mip.BINARY, name=f"z_{u}_{v}") for u, v in edges}

# 3. 目的関数の設定
# カットサイズの総和を最小化
model.objective = mip.xsum(edge_weights[edge] * z[edge] for edge in edges)

# 4. 制約条件の追加
# カット辺の定義
for u, v in edges:
    model += z[(u, v)] >= x[u] - x[v]
    model += z[(u, v)] >= x[v] - x[u]

# 区画サイズの制約 (区画1の頂点数が2または3)
model += mip.xsum(x[i] for i in nodes) >= 2, "p1_size_min"
model += mip.xsum(x[i] for i in nodes) <= 3, "p1_size_max"

# 5. 問題の求解
status = model.optimize()

# 6. 結果の表示と考察
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小カットサイズ: {model.objective_value}")

    partition1_nodes = [i for i in nodes if x[i].x >= 0.99]
    partition0_nodes = [i for i in nodes if x[i].x < 0.01]

    print(f"\n区画1の頂点 ({len(partition1_nodes)}個): {sorted(partition1_nodes)}")
    print(f"区画0の頂点 ({len(partition0_nodes)}個): {sorted(partition0_nodes)}")

    # 7. 結果の可視化
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    pos = nx.spring_layout(G, seed=42)

    node_colors = ['skyblue' if i in partition1_nodes else 'lightcoral' for i in nodes]
    cut_edges = [(u, v) for u, v in edges if z[(u, v)].x >= 0.99]

    plt.figure(figsize=(6, 6))
    nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=500, font_size=12)
    nx.draw_networkx_edges(G, pos, edgelist=cut_edges, edge_color='red', width=2.0, style='dashed')
    plt.title(f"分割結果 (カットサイズ: {model.objective_value:.0f})")
    plt.show()

elif status == mip.OptimizationStatus.INFEASIBLE:
    print("実行不可能: 解は存在しません。")
else:
    print(f"最適化ステータス: {status}")

-----

## 演習問題 2


演習問題1のグラフを区画の頂点数が3となるように二分割し、
最小カットサイズを解答せよ。

### 2\. 数理モデルの定式化

基本的な定式化は演習問題1と同じであるが、区画サイズの制約が変更される。

  * **決定変数, パラメータ, 目的関数**: 演習問題1と同様。

  * **制約条件**

    1.  **カット辺の定義**: 演習問題1と同様。
    2.  **区画サイズの制約**: 区画1の頂点数を厳密に3個にする。
$$
  \sum_{i \in V} x_i = 3
$$
    3.  **変数型制約**: 演習問題1と同様。

### 3\. Python (MIP) による実装

In [ ]:
# 必要なライブラリをインポート
import mip
import networkx as nx
import matplotlib.pyplot as plt
import japanize_matplotlib

# グラフの定義
nodes = [0, 1, 2, 3, 4, 5]
edges = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (2, 4), (3, 4), (2, 5), (3, 5), (4, 5)]
edge_weights = {edge: 1 for edge in edges}

# 1. モデルの作成
model = mip.Model(name="graph_partition_2", sense=mip.MINIMIZE)

# 2. 変数の定義
x = {i: model.add_var(var_type=mip.BINARY, name=f"x_{i}") for i in nodes}
z = {(u, v): model.add_var(var_type=mip.BINARY, name=f"z_{u}_{v}") for u, v in edges}

# 3. 目的関数の設定
model.objective = mip.xsum(edge_weights[edge] * z[edge] for edge in edges)

# 4. 制約条件の追加
# カット辺の定義
for u, v in edges:
    model += z[(u, v)] >= x[u] - x[v]
    model += z[(u, v)] >= x[v] - x[u]

# 区画サイズの制約 (区画1の頂点数が厳密に3)
model += mip.xsum(x[i] for i in nodes) == 3, "p1_size_exact"

# 5. 問題の求解
status = model.optimize()

# 6. 結果の表示と考察
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小カットサイズ: {model.objective_value}")

    partition1_nodes = [i for i in nodes if x[i].x >= 0.99]
    partition0_nodes = [i for i in nodes if x[i].x < 0.01]

    print(f"\n区画1の頂点 ({len(partition1_nodes)}個): {sorted(partition1_nodes)}")
    print(f"区画0の頂点 ({len(partition0_nodes)}個): {sorted(partition0_nodes)}")

    # 7. 結果の可視化
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    pos = nx.spring_layout(G, seed=42)

    node_colors = ['skyblue' if i in partition1_nodes else 'lightcoral' for i in nodes]
    cut_edges = [(u, v) for u, v in edges if z[(u, v)].x >= 0.99]

    plt.figure(figsize=(6, 6))
    nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=500, font_size=12)
    nx.draw_networkx_edges(G, pos, edgelist=cut_edges, edge_color='red', width=2.0, style='dashed')
    plt.title(f"分割結果 (カットサイズ: {model.objective_value:.0f})")
    plt.show()

elif status == mip.OptimizationStatus.INFEASIBLE:
    print("実行不可能: 解は存在しません。")
else:
    print(f"最適化ステータス: {status}")

-----

## 演習問題 3

辺に重みのある以下のグラフで、頂点数6を均等に（各区画3頂点）
二分割し、カットされる辺の重みの合計を最小化せよ。

  * **頂点**: $V = {0, 1, 2, 3, 4, 5}$
  * **辺と重み**: $(0,1,w=3), (0,2,w=1), (0,3,w=2), (1,2,w=2), (1,4,w=4), (2,3,w=5), (2,4,w=1), (2,5,w=2), (3,5,w=3), (4,5,w=2)$

### 2\. 数理モデルの定式化

辺の重みが考慮される点以外は、演習問題2の定式化と同様である。

  * **決定変数, パラメータ**: 演習問題2と同様。ただし、パラメータ $w_{uv}$ は与えられた重みを用いる。

  * **目的関数**
    カットされた辺の重みの合計を最小化する。
$$
 \min \sum_{(u,v) \in E} w_{uv} z_{uv}
$$

  * **制約条件**: 演習問題2と同様。
$$
    \sum_{i \in V} x_i = 3
$$

###Python (MIP) による実装


In [ ]:
# 必要なライブラリをインポート
import mip
import networkx as nx
import matplotlib.pyplot as plt
import japanize_matplotlib

# グラフの定義 (重み付き)
nodes = [0, 1, 2, 3, 4, 5]
edges_with_weights = [
    (0, 1, 3), (0, 2, 1), (0, 3, 2),
    (1, 2, 2), (1, 4, 4),
    (2, 3, 5), (2, 4, 1), (2, 5, 2),
    (3, 5, 3),
    (4, 5, 2)
]
edges = [(u, v) for u, v, w in edges_with_weights]
edge_weights = {(u, v): w for u, v, w in edges_with_weights}

# 1. モデルの作成
model = mip.Model(name="graph_partition_3", sense=mip.MINIMIZE)

# 2. 変数の定義
x = {i: model.add_var(var_type=mip.BINARY, name=f"x_{i}") for i in nodes}
z = {(u, v): model.add_var(var_type=mip.BINARY, name=f"z_{u}_{v}") for u, v in edges}

# 3. 目的関数の設定
# カットされた辺の重みの総和を最小化
model.objective = mip.xsum(edge_weights[edge] * z[edge] for edge in edges)

# 4. 制約条件の追加
# カット辺の定義
for u, v in edges:
    model += z[(u, v)] >= x[u] - x[v]
    model += z[(u, v)] >= x[v] - x[u]

# 区画サイズの制約 (区画1の頂点数が厳密に3)
model += mip.xsum(x[i] for i in nodes) == 3, "p1_size_exact"

# 5. 問題の求解
status = model.optimize()

# 6. 結果の表示と考察
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小カットサイズ（重み合計）: {model.objective_value}")

    partition1_nodes = [i for i in nodes if x[i].x >= 0.99]
    partition0_nodes = [i for i in nodes if x[i].x < 0.01]

    print(f"\n区画1の頂点 ({len(partition1_nodes)}個): {sorted(partition1_nodes)}")
    print(f"区画0の頂点 ({len(partition0_nodes)}個): {sorted(partition0_nodes)}")

    # 7. 結果の可視化
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_weighted_edges_from(edges_with_weights)
    pos = nx.spring_layout(G, seed=42)

    node_colors = ['skyblue' if i in partition1_nodes else 'lightcoral' for i in nodes]
    cut_edges = [(u, v) for u, v in edges if z[(u, v)].x >= 0.99]

    plt.figure(figsize=(8, 8))
    nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=500, font_size=12)
    nx.draw_networkx_edges(G, pos, edgelist=cut_edges, edge_color='red', width=2.0, style='dashed')
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
    plt.title(f"分割結果 (カット重み合計: {model.objective_value:.0f})")
    plt.show()

elif status == mip.OptimizationStatus.INFEASIBLE:
    print("実行不可能: 解は存在しません。")
else:
    print(f"最適化ステータス: {status}")

-----

## 演習問題 4

演習問題1のグラフと条件で、頂点2と頂点4が必ず異なる区画に属するように制約を加えて二分割し、カットサイズを最小化せよ。

### 数理モデルの定式化

演習問題1の定式化に、新しい制約を1つ追加する。

  * **決定変数, パラメータ, 目的関数**: 演習問題1と同様。

  * **制約条件**

   1.  **カット辺の定義**: 演習問題1と同様。
   2.  **区画サイズの制約**: 演習問題1と同様 ($2 \le \sum x_i \le 3$)。
   3.  **変数型制約**: 演習問題1と同様。
   4.  **特定頂点の分離制約**: 頂点2と頂点4が異なる区画に属さなければならない。これは、一方の頂点が区画1 (x=1) にあれば、もう一方は区画0 (x=0) にあることを意味する。これを線形制約で表現すると以下のようになる。
$$
      x_2 + x_4 = 1
$$

###Python (MIP) による実装


In [ ]:
# 必要なライブラリをインポート
import mip
import networkx as nx
import matplotlib.pyplot as plt
import japanize_matplotlib

# グラフの定義
nodes = [0, 1, 2, 3, 4, 5]
edges = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (2, 4), (3, 4), (2, 5), (3, 5), (4, 5)]
edge_weights = {edge: 1 for edge in edges}

# 1. モデルの作成
model = mip.Model(name="graph_partition_4", sense=mip.MINIMIZE)

# 2. 変数の定義
x = {i: model.add_var(var_type=mip.BINARY, name=f"x_{i}") for i in nodes}
z = {(u, v): model.add_var(var_type=mip.BINARY, name=f"z_{u}_{v}") for u, v in edges}

# 3. 目的関数の設定
model.objective = mip.xsum(edge_weights[edge] * z[edge] for edge in edges)

# 4. 制約条件の追加
# カット辺の定義
for u, v in edges:
    model += z[(u, v)] >= x[u] - x[v]
    model += z[(u, v)] >= x[v] - x[u]

# 区画サイズの制約 (区画1の頂点数が2または3)
model += mip.xsum(x[i] for i in nodes) >= 2, "p1_size_min"
model += mip.xsum(x[i] for i in nodes) <= 3, "p1_size_max"

# ★追加の制約: 頂点2と4は異なる区画に属する
model += x[2] + x[4] == 1, "separate_nodes"

# 5. 問題の求解
status = model.optimize()

# 6. 結果の表示と考察
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小カットサイズ: {model.objective_value}")

    partition1_nodes = [i for i in nodes if x[i].x >= 0.99]
    partition0_nodes = [i for i in nodes if x[i].x < 0.01]

    print(f"\n区画1の頂点 ({len(partition1_nodes)}個): {sorted(partition1_nodes)}")
    print(f"区画0の頂点 ({len(partition0_nodes)}個): {sorted(partition0_nodes)}")
    print(f"頂点2は区画{'1' if 2 in partition1_nodes else '0'}に、頂点4は区画{'1' if 4 in partition1_nodes else '0'}に属します。")


    # 7. 結果の可視化
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    pos = nx.spring_layout(G, seed=42)

    node_colors = ['skyblue' if i in partition1_nodes else 'lightcoral' for i in nodes]
    cut_edges = [(u, v) for u, v in edges if z[(u, v)].x >= 0.99]

    plt.figure(figsize=(6, 6))
    nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=500, font_size=12)
    nx.draw_networkx_edges(G, pos, edgelist=cut_edges, edge_color='red', width=2.0, style='dashed')
    plt.title(f"分割結果 (カットサイズ: {model.objective_value:.0f})")
    plt.show()

elif status == mip.OptimizationStatus.INFEASIBLE:
    print("実行不可能: 解は存在しません。")
else:
    print(f"最適化ステータス: {status}")

-----

## 演習問題 5

演習問題1のグラフ（頂点数6）を、3つの区画に分割することを考える。
3つの区画にそれぞれ2頂点ずつ均等に分割する。
異なる区画間のカットサイズを最小化し、カットサイズを解答せよ。


###数理モデルの定式化

グラフを $k=3$ 個の区画に分割する問題として定式化する。

  * **決定変数**
   * $x_{ip}$: 頂点 $i \in V$ が区画 $p \in \{0, 1, 2\}$ に属するならば1、そうでなければ0をとるバイナリ変数。
   * $z_{uv}$: 辺 $(u,v) \in E$ がカットされるならば1、そうでなければ0をとるバイナリ変数。

  * **パラメータ**: 演習問題1と同様。

  * **目的関数**: カットサイズの最小化。

$$
  \min \sum_{(u,v) \in E} z_{uv}
$$

  * **制約条件**

    1.  **頂点の所属制約**: 各頂点は、必ずいずれか1つの区画に属さなければならない。
$$
        \sum_{p=0}^{2} x_{ip} = 1 \quad \forall i \in V
$$

    2.  **カット辺の定義**: 頂点 $u$ と $v$ が異なる区画に属する場合に $z_{uv}=1$ となるようにする。これは以下の制約で表現できる。
$$
\begin{align}
        z_{uv} &\ge x_{up} - x_{vp} \quad & \forall (u,v) \in E, \forall p \in \{0, 1, 2\} \\
        z_{uv} &\ge x_{vp} - x_{up} \quad & \forall (u,v) \in E, \forall p \in \{0, 1, 2\}
\end{align}
$$

    3.  **区画サイズの制約**: 各区画の頂点数を2個にする。
$$
\begin{align}
        \sum_{i \in V} x_{i0} & = 2 \\
        \sum_{i \in V} x_{i1} & = 2 \\
        \sum_{i \in V} x_{i2} & = 2
\end{align}
$$
    4.  **変数型制約**: 各変数がバイナリ変数であることを定義する。

### Python (MIP) による実装


In [ ]:
# 必要なライブラリをインポート
import mip
import networkx as nx
import matplotlib.pyplot as plt
import japanize_matplotlib

# グラフの定義
nodes = [0, 1, 2, 3, 4, 5]
edges = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (2, 4), (3, 4), (2, 5), (3, 5), (4, 5)]
edge_weights = {edge: 1 for edge in edges}

# 分割数と区画
num_partitions = 3
partitions = range(num_partitions)

# 1. モデルの作成
model = mip.Model(name="graph_partition_multi", sense=mip.MINIMIZE)

# 2. 変数の定義
# x_ip: 頂点iが区画pに属するなら1
x = {(i, p): model.add_var(var_type=mip.BINARY, name=f"x_{i}_{p}") for i in nodes for p in partitions}
# z_uv: 辺(u,v)がカットされるなら1
z = {(u, v): model.add_var(var_type=mip.BINARY, name=f"z_{u}_{v}") for u, v in edges}

# 3. 目的関数の設定
model.objective = mip.xsum(z[edge] for edge in edges)

# 4. 制約条件の追加
# 各頂点はちょうど1つの区画に属する
for i in nodes:
    model += mip.xsum(x[(i, p)] for p in partitions) == 1, f"node_assign_{i}"

# カット辺の定義
for u, v in edges:
    for p in partitions:
        model += z[(u, v)] >= x[(u, p)] - x[(v, p)]
        model += z[(u, v)] >= x[(v, p)] - x[(u, p)]

# 区画サイズの制約 (各区画の頂点数が2)
for p in partitions:
    model += mip.xsum(x[(i, p)] for i in nodes) == 2, f"partition_size_{p}"

# 5. 問題の求解
status = model.optimize()

# 6. 結果の表示と考察
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小カットサイズ: {model.objective_value}")

    node_to_partition = {i: p for i in nodes for p in partitions if x[(i, p)].x >= 0.99}

    for p in partitions:
        partition_nodes = [i for i, part in node_to_partition.items() if part == p]
        print(f"区画{p}の頂点: {sorted(partition_nodes)}")

    # 7. 結果の可視化
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges)
    pos = nx.spring_layout(G, seed=10)

    color_map = ['skyblue', 'lightcoral', 'lightgreen']
    node_colors = [color_map[node_to_partition[i]] for i in nodes]
    cut_edges = [(u, v) for u, v in edges if z[(u, v)].x >= 0.99]

    plt.figure(figsize=(6, 6))
    nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=500, font_size=12)
    nx.draw_networkx_edges(G, pos, edgelist=cut_edges, edge_color='red', width=2.0, style='dashed')
    plt.title(f"分割結果 (カットサイズ: {model.objective_value:.0f})")
    plt.show()

elif status == mip.OptimizationStatus.INFEASIBLE:
    print("実行不可能: 解は存在しません。")
else:
    print(f"最適化ステータス: {status}")


## 演習問題6

演習問題3のグラフと条件（各区画3頂点、重み付き）で、頂点0, 頂点1, 頂点2が必ず同じ区画に属するように制約を加えて2分割し、カットされる辺の重みの合計を最小化せよ。

###数理モデルの定式化

演習問題3のモデルに、特定の頂点を同一区画にまとめる制約を追加します。

**決定変数・目的関数**
演習問題3と同様です。

**制約条件**

1.  **カット辺の定義**: 演習問題3と同様。
2.  **区画サイズの制約**: 演習問題3と同様。
$$
  \sum_{i \in V} x_i = 3
$$

3.  **特定頂点の同一区画制約**: 頂点0, 1, 2が同じ区画に属すことを強制します。これは、$x_0, x_1, x_2$ の値が全て等しいことを意味します。
$$
  x_0 = x_1,  \quad x_1 = x_2
$$

4.  **変数型制約**: 演習問題3と同様。

###Python (MIP) による実装


In [ ]:
# 必要なライブラリをインポート
import mip
import networkx as nx
import matplotlib.pyplot as plt
import japanize_matplotlib

# 1. グラフの定義と問題設定
nodes = [0, 1, 2, 3, 4, 5]
edges_with_weights = [
    (0, 1, 3), (0, 2, 1), (0, 3, 2),
    (1, 2, 2), (1, 4, 4),
    (2, 3, 5), (2, 4, 1), (2, 5, 2),
    (3, 5, 3),
    (4, 5, 2)
]
edges = [(u, v) for u, v, w in edges_with_weights]
edge_weights = {(u, v): w for u, v, w in edges_with_weights}

# 区画サイズの制約
partition1_size = 3

# 2. モデルの作成
model = mip.Model(name="graph_partition_6", sense=mip.MINIMIZE)

# 3. 変数の定義
x = {i: model.add_var(var_type=mip.BINARY, name=f"x_{i}") for i in nodes}
z = {(u, v): model.add_var(var_type=mip.BINARY, name=f"z_{u}_{v}") for u, v in edges}

# 4. 目的関数の設定
model.objective = mip.xsum(edge_weights[(u, v)] * z[(u, v)] for u, v in edges)

# 5. 制約条件の追加
# カット辺の定義
for u, v in edges:
    model += z[(u, v)] >= x[u] - x[v]
    model += z[(u, v)] >= x[v] - x[u]

# 区画サイズの制約
model += mip.xsum(x[i] for i in nodes) == partition1_size

# ★追加の制約: 頂点0, 1, 2は同じ区画に属する
model += x[0] == x[1]
model += x[1] == x[2]

# 6. 問題の求解
status = model.optimize()

# 7. 結果の表示と可視化
if status == mip.OptimizationStatus.OPTIMAL:
    print(f"最小カットサイズ（重み合計）: {model.objective_value:.0f}")

    partition1_nodes = [i for i in nodes if x[i].x >= 0.99]
    partition0_nodes = [i for i in nodes if x[i].x < 0.01]

    print(f"\n区画1の頂点: {sorted(partition1_nodes)}")
    print(f"区画0の頂点: {sorted(partition0_nodes)}")

    # グラフの作成
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_weighted_edges_from(edges_with_weights)

    # 可視化
    pos = nx.spring_layout(G, seed=42)
    node_colors = ['skyblue' if n in partition1_nodes else 'lightcoral' for n in G.nodes()]
    cut_edges = [(u, v) for u, v in edges if z[(u, v)].x >= 0.99]

    plt.figure(figsize=(8, 8))
    nx.draw(G, pos, with_labels=True, node_color=node_colors, node_size=500, font_size=12)
    nx.draw_networkx_edges(G, pos, edgelist=cut_edges, edge_color='red', width=2.5)
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
    plt.title(f"分割結果 (カット重み合計: {model.objective_value:.0f})")
    plt.show()

else:
    print(f"最適化ステータス: {status}")